In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# --- 1. Abrir imagen multibanda (VV y VH) ---
ruta = "muestras.tif"

with rasterio.open(ruta) as src:
    img = src.read()  # (bands, rows, cols)
    profile = src.profile

# Transponer a (rows, cols, bands)
img = np.transpose(img, (1, 2, 0))  # (H, W, B)

# --- 2. Simular datos de entrenamiento ---
# Seleccionar algunas ubicaciones manualmente
# Ejemplo: zonas centrales para clase 1 (vegetación), esquinas para clase 0 (suelo)

h, w, b = img.shape
X_train = []
y_train = []

# Vegetación (VV alto, VH medio)
for i in range(30, 50):
    for j in range(30, 50):
        X_train.append(img[i, j])
        y_train.append(1)  # Vegetación

# Suelo (VV y VH bajos)
for i in range(0, 20):
    for j in range(0, 20):
        X_train.append(img[i, j])
        y_train.append(0)  # Suelo

X_train = np.array(X_train)
y_train = np.array(y_train)

# --- 3. Entrenar modelo con XGBoost ---
model = XGBClassifier(n_estimators=100, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# --- 4. Clasificar toda la imagen ---
flat_img = img.reshape(-1, b)  # (H*W, B)
valid_mask = np.all(flat_img > 0, axis=1)  # Excluir píxeles con 0s
valid_pixels = flat_img[valid_mask]

# Predecir
predicted = model.predict(valid_pixels)

# --- 5. Reconstruir imagen clasificada ---
label_map = np.full(flat_img.shape[0], -1, dtype=np.int16)
label_map[valid_mask] = predicted

classified_image = label_map.reshape(h, w)

# --- 6. Guardar imagen clasificada ---
profile.update(dtype='uint8', count=1)

with rasterio.open("clasificacion_xgboost.tif", 'w', **profile) as dst:
    dst.write(classified_image.astype('uint8'), 1)

# --- 7. Visualizar ---
plt.figure(figsize=(10, 6))
plt.imshow(classified_image, cmap='terrain')
plt.title("Clasificación con XGBoost (0=Suelo, 1=Vegetación)")
plt.axis('off')
plt.colorbar()
plt.show()
